In [1]:
from pydantic import BaseModel, Field
from typing import List, Optional
import utils
from importlib import reload


class Run(BaseModel):
    run_number: int
    report: str
    embeddings: Optional[List[float]] = None

class Report(BaseModel):
    name: str
    run: List[Run]
    condition: List[str]

# Sub-model for an individual character in the dream.
class Character(BaseModel):
    number: int = Field(..., description="An identifier or order number for the character in the dream.")
    gender: Optional[str] = Field(..., description="The gender of the character (e.g., 'male', 'female', etc.).")
    identity: Optional[str] = Field(..., description="The role or relationship of the character (e.g., 'self', 'mother', 'stranger').")
    age: Optional[int] = Field(..., description="The age of the character if known.")

# Model for grouping all characters.
class Characters(BaseModel):
    characters: List[Character] = Field(..., description="A list of all characters present in the dream.")

# Model for coding social interactions.
class SocialInteractions(BaseModel):
    aggression: Optional[float] = Field(..., description="A score indicating the level or frequency of aggressive interactions.")
    friendliness: Optional[float] = Field(..., description="A score indicating the level or frequency of friendly interactions.")
    sexuality: Optional[float] = Field(..., description="A score indicating the presence or intensity of sexual interactions.")

# Model for coding different types of activities.
class Activities(BaseModel):
    physical: Optional[float] = Field(..., description="A measure of the physical activity in the dream.")
    movement: Optional[float] = Field(..., description="A measure of the intensity or frequency of movement.")
    location_change: Optional[float] = Field(..., description="A measure of how often the dream location changes.")
    verbal: Optional[float] = Field(..., description="A measure of verbal communication in the dream.")
    expressive_communication: Optional[float] = Field(..., description="A measure of non-verbal expressive communication.")
    visual: Optional[float] = Field(..., description="A measure of the vividness or intensity of visual imagery.")
    auditory: Optional[float] = Field(..., description="A measure of the clarity or intensity of auditory elements.")
    thinking: Optional[float] = Field(..., description="A measure of cognitive activity or internal dialogue.")

# Model for coding emotions present in the dream.
class Emotions(BaseModel):
    anger: Optional[float] = Field(..., description="A score representing the intensity or frequency of anger in the dream.")
    apprehension: Optional[float] = Field(..., description="A score representing the level of apprehension experienced.")
    sadness: Optional[float] = Field(..., description="A score representing the degree of sadness.")
    confusion: Optional[float] = Field(..., description="A score representing the level of confusion in the dream.")
    happiness: Optional[float] = Field(..., description="A score representing the intensity or presence of happiness.")

# Model for coding the setting of the dream.
class Settings(BaseModel):
    location: Optional[str] = Field(..., description="A textual description of the dream's setting or location.")
    indoor: Optional[bool] = Field(..., description="Indicates if the dream setting is indoors.")
    outdoor: Optional[bool] = Field(..., description="Indicates if the dream setting is outdoors.")
    ambiguous: Optional[bool] = Field(..., description="Indicates if the setting is ambiguous or unclear.")
    no_setting: Optional[bool] = Field(..., description="Indicates if no clear setting is present in the dream.")
    familiarity: Optional[str] = Field(
        ..., 
        description="A descriptor indicating the familiarity of the setting (e.g., 'familiar', 'distorted', 'geographical', 'unfamiliar', or 'questionable')."
    )
    familiar: Optional[bool] = Field(..., description="Indicates if the setting is recognized as familiar.")
    distorted: Optional[bool] = Field(..., description="Indicates if the setting appears distorted.")
    geographical: Optional[bool] = Field(..., description="Indicates if the setting is geographically coherent.")
    unfamiliar: Optional[bool] = Field(..., description="Indicates if the setting is unfamiliar.")
    questionable: Optional[bool] = Field(..., description="Indicates if the nature of the setting is questionable.")

# The overarching model for a dream report.
class DreamReport(BaseModel):
    description: str = Field(..., description="The raw textual description of the dream.")
    characters: Characters = Field(..., description="The coded list of characters in the dream.")
    social_interactions: SocialInteractions = Field(..., description="The coded social interactions present in the dream.")
    activities: Activities = Field(..., description="The coded activities and actions performed in the dream.")
    emotions: Emotions = Field(..., description="The coded emotional content of the dream.")
    settings: Settings = Field(..., description="The coded details of the dream's setting.")

In [2]:
import pickle
embedded_reports = "data/interim/reports.pkl"
with open(embedded_reports, "rb") as f:
    report_data = pickle.load(f)
reports = [Report(**data) for data in report_data]
example_reports = [reports[0].run[2].report, reports[3].run[2].report]

In [10]:

reload(utils)
example_dreams = await utils.batch_generate_completions(texts=example_reports, agent="interpreter", response_model=DreamReport, model="gpt-40")

ChatCompletion(id='chatcmpl-BBpPmO5RQFDstToFRmooH6XAbPZ58', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "Description": "The dream involves interactions with employees and staff members.",\n  "Characters": [\n    {\n      "number": 1,\n      "gender": null,\n      "identity": "employee",\n      "age": null\n    },\n    {\n      "number": 2,\n      "gender": null,\n      "identity": "staff member",\n      "age": null\n    }\n  ],\n  "Social Interactions": {\n    "aggression": "low",\n    "friendliness": "moderate",\n    "sexuality": "none"\n  },\n  "Activities": {\n    "physical": "low",\n    "movement": "low",\n    "location_change": "none",\n    "verbal": "moderate",\n    "expressive_communication": "low",\n    "visual": "low",\n    "auditory": "low",\n    "thinking": "moderate"\n  },\n  "Emotions": {\n    "anger": "none",\n    "apprehension": "low",\n    "sadness": "none",\n    "confusion": "low",\n    "happiness":

In [11]:
example_dreams

[DreamReport(description='The dream involves interactions with employees and staff.', characters=Characters(characters=[Character(number=1, gender=None, identity='employee', age=None), Character(number=2, gender=None, identity='staff', age=None)]), social_interactions=SocialInteractions(aggression=None, friendliness=None, sexuality=None), activities=Activities(physical=None, movement=None, location_change=None, verbal=None, expressive_communication=None, visual=None, auditory=None, thinking=None), emotions=Emotions(anger=None, apprehension=None, sadness=None, confusion=None, happiness=None), settings=Settings(location=None, indoor=None, outdoor=None, ambiguous=None, no_setting=None, familiarity=None, familiar=None, distorted=None, geographical=None, unfamiliar=None, questionable=None)),
 DreamReport(description="The dream involves a scenario related to the TV series 'Hubert und Staller', where the dreamer participates in a traffic test that includes a multiple-choice questionnaire.", c